In [1]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

Python 3.10.12


In [2]:
from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')
from mediapipe_model_maker import object_detector
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Mounted at /content/drive


In [3]:
train_data=object_detector.Dataset.from_pascal_voc_folder("drive/My Drive/Colab Notebooks/train")
validation_data = object_detector.Dataset.from_pascal_voc_folder("drive/My Drive/Colab Notebooks/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

train_data size:  200
validation_data size:  22


In [4]:
spec = object_detector.SupportedModels.MOBILENET_V2

hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [5]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   149056    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


25/25 [==============================] - 115s 3s/step - total_loss: 4.7241 - cls_loss: 4.2643 - box_loss: 0.0081 - model_loss: 4.6692 - val_total_loss: 1.4963 - val_cls_loss: 1.0850 - val_box_loss: 0.0071 - val_model_loss: 1.4413
Epoch 2/30
25/25 [==============================] - 67s 3s/step - total_loss: 1.3182 - cls_loss: 0.9922 - box_loss: 0.0054 - model_loss: 1.2632 - val_total_loss: 1.1591 - val_cls_loss: 0.9063 - val_box_loss: 0.0040 - val_model_loss: 1.1041
Epoch 3/30
25/25 [==============================] - 66s 3s/step - total_loss: 0.9619 - cls_loss: 0.7406 - box_loss: 0.0033 - model_loss: 0.9069 - val_total_loss: 0.8635 - val_cls_loss: 0.6563 - val_box_loss: 0.0030 - val_model_loss: 0.8085
Epoch 4/30
25/25 [==============================] - 69s 3s/step - total_loss: 0.6767 - cls_loss: 0.5100 - box_loss: 0.0022 - model_loss: 0.6218 - val_total_loss: 0.7358 - val_cls_loss: 0.5368 - val_box_loss: 0.0029 - val_model_loss: 0.6808
Epoch 5/30
25/25 [==============================] 

In [6]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

6/6 [==============================] - 3s 408ms/step - total_loss: 0.5880 - cls_loss: 0.4137 - box_loss: 0.0024 - model_loss: 0.5331
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.19s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.536
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.097
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.536
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.277
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.429
 Average Recall     (AR) @[ Io

In [7]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	float_ckpt.index  metadata.json  model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>